## Dealing with Complex Images (Self-Defined Datasets)

#### Author : ishandeveloper

In previous examples we built an image classifier that worked using a deep neural network and saw how to improve its performance by adding convolutions. One limitation though was that it used a dataset of very uniform images. Images of clothing that was staged and framed in 28 by 28. 

But what happens when we use larger images and where the feature might be in different locations? For example, how about a few images of horses and humans? They have different sizes and different aspect ratios. The subject can be in different locations. In some cases, there may even be multiple subjects. In addition to that, the earlier examples with a fashion data used a built-in dataset. All of the data was handily split into training and test sets for us and labels were available. In many scenarios, that's not going to be the case and we'll have to do it for yourself.

So, To make it easier for us we'll use Tensorflow APIs and Take a look at the <br>
### <b>Image generator in TensorFlow</b>

Let's Jump right into the code to see it in action.

Getting Dependencies

In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy

First of all, we have to import the <b>ImageDataGenerator</b> from tensorflow library

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We can then instantiate an image generator like this and pass rescale to it to normalize the data. We can then call the flow from directory method on it to get it to load images from that directory and its sub-directories.

The nice thing about this code is that the <b>images are resized</b> for you as they're loaded. So you don't need to preprocess thousands of images on your file system.The images will be loaded for training and validation in <b>batches</b> where it's more efficient than doing it one by one.Finally, there's the class mode. 

Now, this is a <b>binary classifier</b> i.e. it picks between two different things;.

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(300,300),
                batch_size=128,
                class_mode='binary' )

Doing the same steps for the test data

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
                validation_dir,
                target_size=(300,300),
                batch_size=128,
                class_mode='binary' )

So here's the code. As you can see, it's the sequential as before with convolutions and pooling before we get to the dense layers at the bottom. 

But there are a few differences. 

First of all, there are <b>three sets of convolution pooling layers</b> at the top. This reflects the higher complexity and size of the images.

Another thing to pay attention to is the input shape. We resize their images to be 300 by 300 as they were loaded, but they're also color images. So there are three bytes per pixel. One for R,G,B each and that's a common 24-bit color pattern. 

Now, The output layer has also changed. Because previously, we had one neuron per class, but now there's only one neuron for two classes. That's because we're using a different activation function where <b>sigmoid</b> is great for binary classification, where one class will tend towards zero and the other class tending towards one

In [8]:
model = keras.Sequential([
    keras.layers.Conv2D(16, (3,3), activation='relu',input_shape=(300, 300, 3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(512, activation=tf.nn.relu ),
    keras.layers.Dense(1,  activation=tf.nn.sigmoid)
]) 

Now, if we take a look at our model summary, we can see the journey of the image data through the convolutions The 300 by 300 becomes 298 by 298 after the three by three filter, it gets pulled to 149 by 149 which in turn gets reduced to 73 by 73 after the filter that then gets pulled to 35 by 35, this will then get flattened, so 64 convolutions that are 35 squared and shape will get fed into the DNN. If you multiply 35 by 35 by 64, you get 78,400, and that's the shape of the data once it comes out of the convolutions. If we had just fed raw 300 by 300 images without the convolutions, that would be over 900,000 values. So we've already reduced it quite a bit.

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 78400)             0

Now compile the model and, as always, we have a loss function and an optimizer. 

When classifying the ten items of fashion, we might remember that our loss function was a categorical cross entropy. But because we're doing a binary choice here, let's pick a <b>binary_crossentropy</b> instead. Also, earlier we used an Adam optimizer but we can also use RMSprop, where we can adjust the learning rate to experiment with performance.

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['acc'])

Now that we’ve designed the neural network to classify Horses or Humans, the next step is to train it from data that’s on the file system, which can be read by generators. To do this, <b>we don’t use model.fit</b> as earlier, but a new method call: <b>model.fit_generator</b>.